In [1]:
%matplotlib inline

import io, os, sys, types, datetime, pickle, warnings

warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float, filters, morphology, transform

from sklearn import linear_model
from sklearn import metrics

In [2]:
""" ============== path settings =============="""

' ============== path settings =============='

In [3]:
module_path = os.path.join(os.path.dirname(os.getcwd()), 'python_cluster', 'helper_functions')

In [4]:
# env = 'Windows'
# if(env == 'Windows'):
#     module_path = 'W:\\2019_09_NSP_Extension\\code\\NSP_codes\\python_cluster\\helper_functions'
# elif(env == 'Mac'):
#     module_path = '/Users/lily/Lily/Academic/AW_Lab/code/python_cluster/helper_functions'

In [5]:
# Mac, Data_Output, Figure_Output, Ctrl_22hrs_Gal80_s1r1_annotation.csv, 0, 1, 24, 18, 40, 20, 1

In [7]:
### import custom functions
if module_path not in sys.path:
    sys.path.append(module_path)
import data_quantification_settings as settings
import data_quantification_function_helper as my_help
import data_quantification_function_intensity_calculation as my_int
import data_quantification_function_parse_bundle as my_pb
import data_quantification_function_plotting as my_plot

Mac, Data_Output, Figure_Output, Ctrl_22hrs_Gal80_s1r1_annotation.csv, 0, 1, 24, 18, 40, 20, 1
Mac, Data_Output, Figure_Output, Ctrl_22hrs_Gal80_s1r1_annotation.csv, 0, 1, 24, 18, 40, 20, 1


In [8]:
paths = settings.paths
matching_info = settings.matching_info
analysis_params_general = settings.analysis_params_general

In [98]:
analysis_params_general.add_col_params({
    'id_cols':['CategoryID', 'SampleID', 'RegionID', 'TimeID'],
    'dist_cols':[
        'dT0T4',
        'dT0T5',
        'dT4T2',
        'dT4T5',
        'dT4T3',
        'dT4T7',
        'dT2T3',
        'dT5T7',
        'dR0T4']
})

In [97]:
analysis_params_general.add_col_params({})

In [11]:
"""============== main =============="""

'============== main =============='

In [12]:
# # id_cols = [
# #    'CategoryID',
# #     'SampleID',
# #     'RegionID',
# #     'TimeID',
# # ]
# # target_dis_cols = ['dT0T2', 'dT0T5', 'dT4T2', 'dT4T5', 'dT4T3', 'dT4T7', 'dT0T4', 'dT2T3', 'dT5T7', 'dR0T4']
# df_dis_cols = id_cols + ['Bundle_No'] + target_dis_cols

In [99]:
sns.set_style("dark")

In [100]:
def parse_annotation_info(annot_df, roi_df, m2p_ratio):
    is_extended_target_list = False
    annotation_type = annot_df.loc[0,'Annotation_type']
    if(annotation_type == 1):
        bundles_df = my_pb.get_bundles_info_v1(roi_df, annot_df, m2p_ratio[0], m2p_ratio[1], is_extended_target_list)
    if(annotation_type == 2):
        bundles_df = my_pb.get_bundles_info_v2(roi_df, annot_df, m2p_ratio[0], m2p_ratio[1], is_extended_target_list)
    if(annotation_type == 3):
        bundles_df = my_pb.get_bundles_info_v3(roi_df, annot_df, m2p_ratio[0], m2p_ratio[1], is_extended_target_list)
    annot_bundles_df = bundles_df.dropna(axis=0, how='any', inplace = False)
    annot_bundles_df.sort_index(inplace = True)
    return bundles_df, annot_bundles_df

In [113]:
df_dis_cols = analysis_params_general.col_params['id_cols'] + ['Bundle_No'] + analysis_params_general.col_params['dist_cols']
df_dis_2d = pd.DataFrame(columns = df_dis_cols)
df_dis_3d = pd.DataFrame(columns = df_dis_cols)

In [114]:
annot_path = os.path.join(paths.data_prefix, 'Annotations')
cats, files = my_help.parse_folder_info(annot_path)
for cat in cats:
    print(cat)
    times, files = my_help.parse_folder_info(os.path.join(annot_path, cat))
    for time in times:
        print(time)
        folders, files = my_help.parse_folder_info(os.path.join(annot_path, cat, time))
        pwd = os.path.join(annot_path, cat, time)
        for filename in files:
            print(filename)
            if('.csv' in filename):
                df_tmp = pd.read_csv(os.path.join(pwd, filename))
                df_tmp_2d, df_tmp_3d = get_dist_dics(df_tmp)
                df_dis_2d = df_dis_2d.append(df_tmp_2d, ignore_index=True, sort=True)
                df_dis_3d = df_dis_2d.append(df_tmp_3d, ignore_index=True, sort=True)

Nic
28
Nic_28hrs_Gal80_s12r1_summary.csv
---annot_df---
1
2
9
12
14
18
21
23
25
30
31
35
40
41
52
57
64
66
70
Nic_28hrs_Gal80_s29r2_summary.csv
---annot_df---
1
2
3
6
13
11
58
12
17
15
36
37
39
61
41
Nic_28hrs_Gal80_s2r1_summary.csv
---annot_df---
1
2
12
11
17
22
25
29
35
Nic_28hrs_Gal80_s8r1_summary.csv
---annot_df---
1
2
3
8
15
16
21
22
27
30
32
36
38
40
41
42
43
45
47
48
55
56
61
76
79
65
Nic_28hrs_Gal80_s23r2_summary.csv
---annot_df---
1
2
7
14
17
25
26
37
3
5
8
19
Ctrl
22
Ctrl_22hrs_Gal80_s1r1_annotation.csv


TypeError: join() argument must be str or bytes, not 'float64'

In [109]:
def get_dist_dics(df_tmp):

    id_cols = analysis_params_general.col_params['id_cols']
    target_dis_cols = analysis_params_general.col_params['dist_cols']

    image_list = df_tmp.loc[:,'Image_Name'].unique()
    ROI_list = df_tmp.loc[:,'ROI_Name'].unique()
    i_image = 0
    image_name = image_list[i_image]
    roi_name = ROI_list[i_image]
    df_roi_temp = pd.read_csv(os.path.join(paths.roi_path, roi_name))
    df_roi_temp.rename(columns = {' ':'No'}, inplace = True)
    df_annot_temp = df_tmp.groupby(['Image_Name']).get_group(image_list[i_image]).reset_index(drop = True)
    m2p_ratio = (df_annot_temp.iloc[0]['imgX_pixel']/df_annot_temp.iloc[0]['imgX_um'], df_annot_temp.iloc[0]['imgY_pixel']/df_annot_temp.iloc[0]['imgY_um'])
    df_bundles_temp, df_annot_bundles_temp = parse_annotation_info(df_annot_temp, df_roi_temp, m2p_ratio)

    df_2d = pd.DataFrame(columns = id_cols + ['Bundle_No'] + target_dis_cols)
    df_3d = pd.DataFrame(columns = id_cols + ['Bundle_No'] + target_dis_cols)

    for bundle_no in df_annot_bundles_temp.index:
        df_tmp_2d = pd.DataFrame(columns = df_dis_2d.columns)
        df_tmp_3d = pd.DataFrame(columns = df_dis_3d.columns)
        coord_ori = np.array([
            df_annot_bundles_temp.loc[bundle_no,'coord_X_Center'],
            df_annot_bundles_temp.loc[bundle_no,'coord_Y_Center'],
            df_annot_bundles_temp.loc[bundle_no,'coord_Z_Center']
        ])
        target_inds, target_coords_2d = my_help.get_target_coords(bundle_no, df_bundles_temp, matching_info.index_to_target_id)
        target_inds, target_coords_3d = my_help.get_3d_target_coords(bundle_no, df_bundles_temp, matching_info.index_to_target_id)
        for i in [0,1]:
            target_coords_2d[:,i] = target_coords_2d[:,i]/m2p_ratio[i]
            target_coords_3d[:,i] = target_coords_3d[:,i]/m2p_ratio[i]
            coord_ori[i] = coord_ori[i]/m2p_ratio[i]
        target_coords_3d[:,2] = target_coords_3d[:,2] * 0.125
        coord_ori[2] = coord_ori[2] * 0.125
        df_tmp_2d.loc[0,id_cols[:-1]] = df_annot_bundles_temp.loc[bundle_no, id_cols[:-1]]
        df_tmp_3d.loc[0,id_cols[:-1]] = df_annot_bundles_temp.loc[bundle_no, id_cols[:-1]]
        df_tmp_2d.loc[0, 'TimeID'] = 28
        df_tmp_3d.loc[0, 'TimeID'] = 28
        df_tmp_2d.loc[0,'Bundle_No'] = bundle_no
        df_tmp_3d.loc[0,'Bundle_No'] = bundle_no
        for cat in target_dis_cols[:-1]:
            ind_i = matching_info.target_id_to_index[int(cat.split('T')[1])]
            ind_j = matching_info.target_id_to_index[int(cat.split('T')[2])]
            df_tmp_2d.loc[0,cat] = np.linalg.norm(target_coords_2d[ind_i,:]-target_coords_2d[ind_j,:])
            df_tmp_3d.loc[0,cat] = np.linalg.norm(target_coords_3d[ind_i,:]-target_coords_3d[ind_j,:])
        df_tmp_2d.loc[0,'dR0T4'] = np.linalg.norm(target_coords_2d[matching_info.target_id_to_index[4],:] - coord_ori[0:2])
        df_tmp_3d.loc[0,'dR0T4'] = np.linalg.norm(target_coords_3d[matching_info.target_id_to_index[4],:] - coord_ori)
        df_2d = df_2d.append(df_tmp_2d, ignore_index=True, sort=True)
        df_3d = df_3d.append(df_tmp_3d, ignore_index=True, sort=True)
    
    return df_2d, df_3d

In [ ]:
# load annotation files
mypath = os.path.join(paths.annot_path, 'Fz', '28')
folders, files = my_help.parse_folder_info(mypath)
for filename in files:
    print(filename)
    if('.csv' in filename):
        df_tmp = pd.read_csv(os.path.join(mypath, filename))
        image_list = df_tmp.loc[:,'Image_Name'].unique()
        ROI_list = df_tmp.loc[:,'ROI_Name'].unique()
        i_image = 0
        image_name = image_list[i_image]
        roi_name = ROI_list[i_image]
        df_roi_temp = pd.read_csv(os.path.join(paths.roi_path, roi_name))
        df_roi_temp.rename(columns = {' ':'No'}, inplace = True)
        df_annot_temp = df_tmp.groupby(['Image_Name']).get_group(image_list[i_image]).reset_index(drop = True)
        m2p_ratio = (df_annot_temp.iloc[0]['imgX_pixel']/df_annot_temp.iloc[0]['imgX_um'], df_annot_temp.iloc[0]['imgY_pixel']/df_annot_temp.iloc[0]['imgY_um'])
        df_bundles_temp, df_annot_bundles_temp = parse_annotation_info(df_annot_temp, df_roi_temp, m2p_ratio)
        
        for bundle_no in df_annot_bundles_temp.index:
            df_tmp_2d = pd.DataFrame(columns = df_dis_2d.columns)
            df_tmp_3d = pd.DataFrame(columns = df_dis_3d.columns)
            coord_ori = np.array([
                df_annot_bundles_temp.loc[bundle_no,'coord_X_Center'],
                df_annot_bundles_temp.loc[bundle_no,'coord_Y_Center'],
                df_annot_bundles_temp.loc[bundle_no,'coord_Z_Center']
            ])
            target_inds, target_coords_2d = my_help.get_target_coords(bundle_no, df_bundles_temp, matching_info.index_to_target_id)
            target_inds, target_coords_3d = my_help.get_3d_target_coords(bundle_no, df_bundles_temp, matching_info.index_to_target_id)
            for i in [0,1]:
                target_coords_2d[:,i] = target_coords_2d[:,i]/m2p_ratio[i]
                target_coords_3d[:,i] = target_coords_3d[:,i]/m2p_ratio[i]
                coord_ori[i] = coord_ori[i]/m2p_ratio[i]
            target_coords_3d[:,2] = target_coords_3d[:,2] * 0.125
            coord_ori[2] = coord_ori[2] * 0.125
            df_tmp_2d.loc[0,id_cols[:-1]] = df_annot_bundles_temp.loc[bundle_no, id_cols[:-1]]
            df_tmp_3d.loc[0,id_cols[:-1]] = df_annot_bundles_temp.loc[bundle_no, id_cols[:-1]]
            df_tmp_2d.loc[0, 'TimeID'] = 28
            df_tmp_3d.loc[0, 'TimeID'] = 28
            df_tmp_2d.loc[0,'Bundle_No'] = bundle_no
            df_tmp_3d.loc[0,'Bundle_No'] = bundle_no
            for cat in target_dis_cols[:-1]:
                ind_i = matching_info.target_id_to_index[int(cat.split('T')[1])]
                ind_j = matching_info.target_id_to_index[int(cat.split('T')[2])]
                df_tmp_2d.loc[0,cat] = np.linalg.norm(target_coords_2d[ind_i,:]-target_coords_2d[ind_j,:])
                df_tmp_3d.loc[0,cat] = np.linalg.norm(target_coords_3d[ind_i,:]-target_coords_3d[ind_j,:])
            df_tmp_2d.loc[0,'R0T4'] = np.linalg.norm(target_coords_2d[matching_info.target_id_to_index[4],:] - coord_ori[0:2])
            df_tmp_3d.loc[0,'R0T4'] = np.linalg.norm(target_coords_3d[matching_info.target_id_to_index[4],:] - coord_ori)
            df_dis_2d = df_dis_2d.append(df_tmp_2d, ignore_index=True, sort=True)
            df_dis_3d = df_dis_2d.append(df_tmp_3d, ignore_index=True, sort=True)

In [ ]:
# load annotation files
mypath = os.path.join(paths.annot_path, 'Nic', '28')
folders, files = my_help.parse_folder_info(mypath)
for filename in files:
    print(filename)
    if('.csv' in filename):
        df_tmp = pd.read_csv(os.path.join(mypath, filename))
        image_list = df_tmp.loc[:,'Image_Name'].unique()
        ROI_list = df_tmp.loc[:,'ROI_Name'].unique()
        i_image = 0
        image_name = image_list[i_image]
        roi_name = ROI_list[i_image]
        df_roi_temp = pd.read_csv(os.path.join(paths.roi_path, roi_name))
        df_roi_temp.rename(columns = {' ':'No'}, inplace = True)
        df_annot_temp = df_tmp.groupby(['Image_Name']).get_group(image_list[i_image]).reset_index(drop = True)
        m2p_ratio = (df_annot_temp.iloc[0]['imgX_pixel']/df_annot_temp.iloc[0]['imgX_um'], df_annot_temp.iloc[0]['imgY_pixel']/df_annot_temp.iloc[0]['imgY_um'])
        df_bundles_temp, df_annot_bundles_temp = parse_annotation_info(df_annot_temp, df_roi_temp, m2p_ratio)
        
        for bundle_no in df_annot_bundles_temp.index:
            df_tmp_2d = pd.DataFrame(columns = df_dis_2d.columns)
            df_tmp_3d = pd.DataFrame(columns = df_dis_3d.columns)
            coord_ori = np.array([
                df_annot_bundles_temp.loc[bundle_no,'coord_X_Center'],
                df_annot_bundles_temp.loc[bundle_no,'coord_Y_Center'],
                df_annot_bundles_temp.loc[bundle_no,'coord_Z_Center']
            ])
            target_inds, target_coords_2d = my_help.get_target_coords(bundle_no, df_bundles_temp, matching_info.index_to_target_id)
            target_inds, target_coords_3d = my_help.get_3d_target_coords(bundle_no, df_bundles_temp, matching_info.index_to_target_id)
            for i in [0,1]:
                target_coords_2d[:,i] = target_coords_2d[:,i]/m2p_ratio[i]
                target_coords_3d[:,i] = target_coords_3d[:,i]/m2p_ratio[i]
                coord_ori[i] = coord_ori[i]/m2p_ratio[i]
            target_coords_3d[:,2] = target_coords_3d[:,2] * 0.125
            coord_ori[2] = coord_ori[2] * 0.125
            df_tmp_2d.loc[0,id_cols[:-1]] = df_annot_bundles_temp.loc[bundle_no, id_cols[:-1]]
            df_tmp_3d.loc[0,id_cols[:-1]] = df_annot_bundles_temp.loc[bundle_no, id_cols[:-1]]
            df_tmp_2d.loc[0, 'TimeID'] = 28
            df_tmp_3d.loc[0, 'TimeID'] = 28
            df_tmp_2d.loc[0,'Bundle_No'] = bundle_no
            df_tmp_3d.loc[0,'Bundle_No'] = bundle_no
            for cat in target_dis_cols[:-1]:
                ind_i = matching_info.target_id_to_index[int(cat.split('T')[1])]
                ind_j = matching_info.target_id_to_index[int(cat.split('T')[2])]
                df_tmp_2d.loc[0,cat] = np.linalg.norm(target_coords_2d[ind_i,:]-target_coords_2d[ind_j,:])
                df_tmp_3d.loc[0,cat] = np.linalg.norm(target_coords_3d[ind_i,:]-target_coords_3d[ind_j,:])
            df_tmp_2d.loc[0,'R0T4'] = np.linalg.norm(target_coords_2d[matching_info.target_id_to_index[4],:] - coord_ori[0:2])
            df_tmp_3d.loc[0,'R0T4'] = np.linalg.norm(target_coords_3d[matching_info.target_id_to_index[4],:] - coord_ori)
            df_dis_2d = df_dis_2d.append(df_tmp_2d, ignore_index=True, sort=True)
            df_dis_3d = df_dis_2d.append(df_tmp_3d, ignore_index=True, sort=True)

In [ ]:
target_dis_cols[:-2]

In [ ]:
df_dis_2d = df_dis_2d.reindex(df_dis_cols, axis=1)
df_dis_3d = df_dis_3d.reindex(df_dis_cols, axis=1)

In [ ]:
df_dis_2d.loc[:,'unit_mean'] = np.mean(df_dis_2d.loc[:,target_dis_cols[:-2]], axis = 1)
df_dis_3d.loc[:,'unit_mean'] = np.mean(df_dis_3d.loc[:,target_dis_cols[:-2]], axis = 1)

In [ ]:
target_dis_cols.append('unit_mean')

In [ ]:
target_dis_cols

In [ ]:
df_dis_2d.columns

In [ ]:
df_dis_2d_reshape = pd.DataFrame(columns = id_cols + ['Bundle_No'] + ['DistCat', 'DistVal'])
df_dis_3d_reshape = pd.DataFrame(columns = id_cols + ['Bundle_No'] + ['DistCat', 'DistVal'])

In [ ]:
target_dis_cols

In [ ]:
for ind in df_dis_2d.index:
    df_tmp_2d = pd.DataFrame(columns = df_dis_2d_reshape.columns)
    df_tmp_2d.loc[0,id_cols] = df_dis_2d.loc[ind,id_cols]
    df_tmp_2d.loc[0,'Bundle_No'] = df_dis_2d.loc[ind,'Bundle_No']
    for cat in target_dis_cols:
        df_tmp_2d.loc[0,'DistCat'] = cat
        df_tmp_2d.loc[0,'DistVal'] = df_dis_2d.loc[ind,cat]
        df_dis_2d_reshape = df_dis_2d_reshape.append(df_tmp_2d, ignore_index=True, sort=True)

In [ ]:
sns.boxplot(x="DistCat", y="DistVal", hue="CategoryID",
                 data=df_dis_2d_reshape)

In [ ]:
""" Process images """
### number of channels
nChannels = min(image.shape)# number of channels of original image
num_norm_channels = len(matching_info.channel_cmap.keys()) # number of channels of normalized image

if(nChannels == 2):
    print("2 channels!")
    ### normalize channels
    image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,0,:,:].dtype, order='C')
    thr = np.zeros((2))
    
    # RFP_norm
    %time image_norm[:,:,:,0] = exposure.rescale_intensity(image[:,0,:,:], in_range = 'image', out_range='dtype')
    # GFP_norm
    %time image_norm[:,:,:,1] = exposure.rescale_intensity(image[:,1,:,:], in_range = 'image', out_range='dtype')    
    
    del image
        
elif(nChannels == 4):
    print("4 channels!")
    ### normalize channels
    image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,:,:,1].dtype, order='C')
    thr = np.zeros((2))
    
    # RFP_norm
    %time image_norm[:,:,:,0] = exposure.rescale_intensity(image[:,:,:,0], in_range = 'image', out_range='dtype')
    # GFP_norm
    %time image_norm[:,:,:,1] = exposure.rescale_intensity(image[:,:,:,1], in_range = 'image', out_range='dtype')    
    

In [ ]:
""" Plot individual bundles """

In [ ]:
bundle_no = 1
plotSettings = False, False, False, True #isPlotR3Line, isPlotR4Line, isPlotR4s, isLabelOff
my_plot.plot_individual_bundles(bundle_no, bundles_df, image_norm, m2p_ratio[0], m2p_ratio[1], is_plot_r3_line = False, is_plot_r4_line = False, is_plot_r4 = False, is_label_off = True)
plt.show()

In [ ]:
for ind, bundle_no in enumerate(annot_bundles_df.index):
    plotSettings = False, False, False, True #isPlotR3Line, isPlotR4Line, isPlotR4s, isLabelOff
    my_plot.plot_individual_bundles(bundle_no, bundles_df, image_norm, m2p_ratio[0], m2p_ratio[1], is_plot_r3_line = False, is_plot_r4_line = False, is_plot_r4 = False, is_label_off = True)
    plt.show()